In [1]:
import numpy as np
import pandas as pd
import sys
import gc
import joblib
import pathlib
import json
import glob
from tqdm.notebook import tqdm

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('seaborn-colorblind')

import plotly.graph_objects as go

In [3]:
df = pd.read_csv("/Users/sandeep/Workspace/APEX/RecoveryStocksArbitrage/Yash_Recovery_Stocks.csv")
df.head()

,Ticker,Name,Total Return:M-6,Rev - 1 Yr Gr:Q,Market Cap,Market Cap.1,Price:D-1,P/E
0,BASE US Equity,COUCHBASE INC,-30.70,20.16,9.150515e+08,9.150515e+08,20.99,NaN
1,SPT US Equity,SPROUT SOCIAL INC - CLASS A,-30.92,45.82,3.414239e+09,3.414239e+09,63.22,NaN
2,ASAN US Equity,ASANA INC - CL A,-31.53,70.34,8.948612e+09,8.948612e+09,48.00,NaN
3,XM US Equity,QUALTRICS INTERNATIONAL-CL A,-32.03,47.98,1.543719e+10,1.543719e+10,27.06,NaN
4,RPAY US Equity,REPAY HOLDINGS CORP,-32.19,62.42,1.552755e+09,1.552755e+09,17.15,NaN
